In [2]:
import os
from gensim.models import KeyedVectors
from collections import Counter
import matplotlib.pyplot as plt

In [3]:
wv = KeyedVectors.load('./coach/model.kv')
file = os.path.abspath('./douban.txt')
print(file)
sentence_vec_list=[]
with open(file,'r') as f:
    for line in f:
        row_vec_list=[]
        for w in line.split(':')[1].split():
            if w in wv:
                row_vec_list.append(wv[w])
        sentence_vec_list.append(row_vec_list)

/bigdata/xiaoma/Assi12/douban.txt


In [4]:
len(sentence_vec_list)

70260

In [5]:
dic = []
for i,article in enumerate(sentence_vec_list):
    dic.append(len(article))

In [6]:
# totals = Counter(dic).most_common(10)
# totals

In [7]:
# sorted(totals)

In [8]:
# dic_totals={}
# totals = Counter(dic).most_common(100)

# totals = sorted(totals)
# for w in totals:
#     dic_totals[w[0]] = w[1]

In [9]:
# X = [str(w) for w in dic_totals.keys()]
# Y = [w for w in dic_totals.values()]
# plt.figure(32)
# plt.bar(X,Y)
# plt.show()

In [10]:
max_length=200

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
comment_id = []
def pad_sequences(sentence_vec_list,maxlen,padding='post',truncating='post'):
    features = np.zeros((len(sentence_vec_list), maxlen,100), dtype=float)    
    for i,sentence_vec in enumerate(sentence_vec_list):
        if not sentence_vec:
            continue
        if len(sentence_vec) <= maxlen and padding == 'pre':
            features[i,-len(sentence_vec):,:] = np.array(sentence_vec)[:,:]
        if len(sentence_vec) <= maxlen and padding == 'post':
            features[i,:len(sentence_vec),:] = np.array(sentence_vec)[:,:]
        if len(sentence_vec) > maxlen and truncating == 'pre':
            features[i,:,:] = np.array(sentence_vec)[:maxlen,:]
        if len(sentence_vec) > maxlen and truncating == 'post':
            features[i,:,:] = np.array(sentence_vec)[len(sentence_vec)-maxlen:,:]   
        comment_id.append(i)
    return features

In [48]:
features = pad_sequences(sentence_vec_list,max_length,padding='post',truncating='post')

In [49]:
Y=[]
with open('label.txt','r',encoding='utf8') as f:
    for line in f:
        Y.append(int(line.strip().split(':')[1]))

In [50]:
print(len(Y))

70260


In [51]:
Y = (np.arange(5) == np.array(Y)[:,None]).astype(np.float32)

In [52]:
Y.shape

(70260, 5)

In [53]:
X = features

In [54]:
X.shape #70260语料   50个长度   100维的词向量 

(70260, 200, 100)

In [55]:
#X = np.reshape(X,(-1,5000))

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size = 0.8)

/home/mk/anaconda3/envs/python3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [57]:
X_train.shape

(56208, 200, 100)

# Model

In [58]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.merge import add
from keras.layers import LSTM,Bidirectional, Dropout,GRU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import regularizers

Using TensorFlow backend.


In [59]:
model = Sequential()

In [ ]:
model.save

In [60]:
# model.add(Embedding(len(word_to_id), embedding_vecor_length, input_length=max_comment_length, dropout=0.2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same',input_shape = (200, 100), activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Bidirectional(LSTM(10)))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax', activity_regularizer=regularizers.l2(0.001)))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 200, 32)           9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 100)          33200     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20)                8880      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)      

In [1]:
history = model.fit(X_train, y_train,validation_split=0.20, epochs=1,verbose=1, batch_size=100)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

from text_cnn import TextCNN

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)